In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install langchain pinecone-client
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 17.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver d

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key = user_secrets.get_secret("PINECONE_API_KEY")

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

index_name = "email-classification"

# initialise with pinecone client
pc = Pinecone(api_key = key)

# create index 
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384, # for dense vector
        metric = "dotproduct", # for sparse value
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [6]:
# Get the index object
index = pc.Index(index_name)
index

In [7]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.3 MB/s eta 0:00:0000:01


In [8]:
import re
import random
from kaggle_secrets import UserSecretsClient
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [9]:
# Initialize Groq LLM
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")
llm = ChatGroq(
    api_key=groq_api_key,
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [10]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 62.0 MB/s eta 0:00:00:00:0100:01


In [11]:
import mysql.connector

def setup_database():
    # MySQL connection parameters
    config = {
        'user': 'root',  # Ensure no leading space
        'password': 'Tanisha1009@',
        'host': '127.0.0.1',  # Use IP address instead of 'localhost'
        'port': 3306,
        'database': 'equipment_rental',
        'raise_on_warnings': True
    }
    
    conn = None
    cursor = None
    
    try:
        # Connect to the MySQL server
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()
        
        # Create tables
        cursor.execute('''CREATE TABLE IF NOT EXISTS equipment
                         (id INT AUTO_INCREMENT PRIMARY KEY, 
                          name VARCHAR(255), 
                          category VARCHAR(255), 
                          price DECIMAL(10, 2), 
                          available BOOLEAN)''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS similar_items
                         (item_id INT, 
                          similar_item_id INT,
                          FOREIGN KEY(item_id) REFERENCES equipment(id),
                          FOREIGN KEY(similar_item_id) REFERENCES equipment(id))''')
        
        # Commit changes
        conn.commit()
        print("Database tables created successfully")
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        
    finally:
        if cursor:
            cursor.close()
        if conn and conn.is_connected():
            conn.close()
            print("MySQL connection is closed")


# Database Fucntions

In [12]:
def check_availability(item_name):
    conn = sqlite3.connect('equipment_rental.db')
    c = conn.cursor()
    c.execute("SELECT available, price FROM equipment WHERE name=?", (item_name,))
    result = c.fetchone()
    conn.close()
    return (result[0], result[1]) if result else (None, None)

def get_similar_items(item_name):
    conn = sqlite3.connect('equipment_rental.db')
    c = conn.cursor()
    c.execute("""SELECT e.name FROM equipment e
                 JOIN similar_items si ON e.id = si.similar_item_id
                 WHERE si.item_id = (SELECT id FROM equipment WHERE name=?)""", (item_name,))
    similar_items = [row[0] for row in c.fetchall()]
    conn.close()
    return similar_items

# Email Classification

In [13]:
def classify_email(email_content):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that classifies emails into categories: inquiry, review, assistance, or other."),
        ("human", "{email}"),
        ("human", "Classify the above email. Respond with only the category name.")
    ])
    chain = prompt | llm
    response = chain.invoke({"email": email_content})
    return response.content.strip().lower()

# Inquiry handling

In [14]:
def handle_inquiry(email_content):
    item_match = re.search(r"about the ([^.?!]+)", email_content, re.IGNORECASE)
    if item_match:
        item_name = item_match.group(1).strip()
        available, price = check_availability(item_name)
        
        if available is not None:
            if available:
                return f"The {item_name} is available for rent at ${price:.2f} per day."
            else:
                similar_items = get_similar_items(item_name)
                if similar_items:
                    items_str = ", ".join(similar_items)
                    return f"Unfortunately, the {item_name} is not available. However, we have similar items you might be interested in: {items_str}."
                else:
                    return f"Unfortunately, the {item_name} is not available, and we don't have any similar items at the moment."
        else:
            return f"I'm sorry, but I couldn't find information about {item_name} in our database. Please check the spelling or inquire about a different item."
    else:
        return "I'm sorry, but I couldn't identify the specific item you're inquiring about. Could you please clarify which piece of equipment you're interested in?"


# Review handling

In [15]:
def analyze_sentiment(review):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that analyzes the sentiment of reviews."),
        ("human", "{review}"),
        ("human", "Analyze the sentiment of the above review. Respond with only 'positive' or 'negative'.")
    ])
    chain = prompt | llm
    response = chain.invoke({"review": review})
    return response.content.strip().lower()

def handle_positive_review(review):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that generates thank you responses for positive reviews."),
        ("human", "{review}"),
        ("human", "Generate a thank you response for the above positive review, encouraging the customer to share their experience on social media.")
    ])
    chain = prompt | llm
    response = chain.invoke({"review": review})
    return response.content.strip()

def handle_negative_review(review):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant that generates apologetic responses for negative reviews."),
        ("human", "{review}"),
        ("human", "Generate an apologetic response for the above negative review, mentioning that a customer service representative will call them soon and offering a gift voucher.")
    ])
    chain = prompt | llm
    response = chain.invoke({"review": review})
    
    voucher_code = f"SORRY-{random.randint(1000, 9999)}"
    escalate_to_crm(review, voucher_code)
    
    return response.content.strip(), voucher_code

def escalate_to_crm(review, voucher_code):
    print(f"Escalated to CRM: Negative review received. Voucher code: {voucher_code}")
    # In a real system, you would integrate with your CRM here


# Assistance request handling

In [16]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.7 MB/s eta 0:00:0000:01


In [17]:
!pip install pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=cc502e44489bea03ba769e7f571e1458d780e80a457ed08f3add4595f0f7d771
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.

In [18]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone_text.sparse import BM25Encoder
from langchain.retrievers import PineconeHybridSearchRetriever
from langchain.chains import RetrievalQA

def setup_rag_pipeline():
    # Sample FAQ content
    faq_content = ["Q: How do I rent equipment? A: To rent equipment, visit our website, select the items you need, choose your rental dates, and complete the checkout process. You can also visit our physical store or call our customer service for assistance.",
    "Q: What is your cancellation policy? A: You can cancel your reservation up to 48 hours before the rental start time for a full refund. Cancellations within 48 hours may be subject to a cancellation fee.",
    "Q: Do you offer insurance for rented equipment? A: Yes, we offer equipment protection plans. These plans cover accidental damage but not loss or theft. You can add this protection during the checkout process.",
    "Q: How do I return the equipment? A: Return the equipment to the same location where you picked it up, during our operating hours. Make sure all items are clean and in the same condition as when you received them.",
    "Q: What if the equipment malfunctions during my rental? A: If you experience any issues with the equipment, contact our customer support immediately. We will either guide you through troubleshooting or arrange for a replacement if necessary.",
    ]
    
    # Use CharacterTextSplitter to split the content
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text("\n".join(faq_content))
    
    # Create dense vectors
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # Create sparse vectors
    bm25_encoder = BM25Encoder().default()
    bm25_encoder.fit(faq_content)
    bm25_encoder.dump("bm25_values.json")
    bm25_encoder = BM25Encoder().load("bm25_values.json")
    
    # Create the hybrid search retriever
    retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)
    
    # Add texts to the index
    retriever.add_texts(faq_content)
    
    # Create the QA chain
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
    
    return qa_chain

# Set up the RAG pipeline
qa_chain = setup_rag_pipeline()


2024-07-27 14:56:03.999133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 14:56:03.999329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 14:56:04.155824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:

def handle_assistance_request(request):
    solution = qa_chain.run(request)
    
    if solution:
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an assistant that generates helpful responses to assistance requests."),
            ("human", "Request: {request}\nSolution: {solution}"),
            ("human", "Generate a helpful response for the above assistance request, incorporating the provided solution.")
        ])
        chain = prompt | llm
        response = chain.invoke({"request": request, "solution": solution})
        return response.content.strip()
    else:
        escalate_to_customer_service(request)
        return "We're sorry, but we couldn't find an immediate solution to your issue. Our customer service team will contact you shortly to assist you further."

def escalate_to_customer_service(request):
    print(f"Escalated to Customer Service: {request}")
    # In a real system, you would integrate with your customer service ticketing system here


# Email sending

In [20]:
def send_email(to_address, subject, body):
    print(f"Sending email to {to_address}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")


In [21]:
# Main processing functions
def process_inquiry(email_content, sender_email):
    response = handle_inquiry(email_content)
    subject = "Response to Your Inquiry"
    send_email(sender_email, subject, response)

def process_review(email_content, sender_email):
    sentiment = analyze_sentiment(email_content)
    if sentiment == "positive":
        response = handle_positive_review(email_content)
        subject = "Thank You for Your Positive Review!"
    else:
        response, voucher_code = handle_negative_review(email_content)
        subject = f"We Value Your Feedback - Gift Voucher {voucher_code}"
    
    send_email(sender_email, subject, response)

def process_assistance_request(email_content, sender_email):
    response = handle_assistance_request(email_content)
    subject = "Response to Your Assistance Request"
    send_email(sender_email, subject, response)

def process_other(email_content, sender_email):
    response = "Thank you for your email. We have forwarded it to our customer service team, and they will get back to you soon."
    subject = "Your Email Has Been Received"
    send_email(sender_email, subject, response)
    print(f"Forwarded to customer service: {email_content}")

def process_email(email_content, sender_email):
    email_type = classify_email(email_content)
    
    if email_type == "inquiry":
        process_inquiry(email_content, sender_email)
    elif email_type == "review":
        process_review(email_content, sender_email)
    elif email_type == "assistance":
        process_assistance_request(email_content, sender_email)
    else:
        process_other(email_content, sender_email)


In [22]:
# Main application
if __name__ == "__main__":
    setup_database()
    
    # Example usage
    emails = [
        ("I'm interested in renting a RED DSMC2 camera. Is it available?", "customer1@example.com"),
        ("The equipment I rented was in excellent condition. Great service!", "customer2@example.com"),
        ("I'm having trouble with the audio mixer. The levels keep fluctuating.", "customer3@example.com"),
        ("When are your business hours?", "customer4@example.com")
    ]
    
    for email_content, sender_email in emails:
        print(f"\nProcessing email from {sender_email}")
        print(f"Content: {email_content}")
        process_email(email_content, sender_email)
        print("--------------------")

    # Cleanup (uncomment if you want to delete the index after testing)
    # pc.delete_index(index_name)

Error: 2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)

Processing email from customer1@example.com
Content: I'm interested in renting a RED DSMC2 camera. Is it available?
Sending email to customer1@example.com
Subject: Response to Your Inquiry
Body: I'm sorry, but I couldn't identify the specific item you're inquiring about. Could you please clarify which piece of equipment you're interested in?
--------------------

Processing email from customer2@example.com
Content: The equipment I rented was in excellent condition. Great service!


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Sending email to customer2@example.com
Subject: Response to Your Assistance Request
Body: Thank you for your positive feedback about the equipment and our service! I'm glad to hear that you had a great experience.

To return the equipment, please bring it back to the original rental location during their business hours. Make sure the equipment is clean and in the same excellent condition as when you received it.

Should you face any issues with the equipment during the rental period, please contact the customer support team right away. They will either assist you with troubleshooting or arrange for a replacement if needed.

Regarding insurance for the rented equipment, the company provides equipment protection plans that cover accidental damage but not loss or theft. If you didn't add this protection during the initial checkout process, it might not be possible to add it retroactively. However, I recommend checking with the rental location for any potential options.

Once again, thank 